# GDL for main graph rewiring and metrics comparison

## Import libraries

In [1]:
import sys
print(sys.executable)
from pathlib import Path
import torch
import torch_geometric
print(torch.__version__)
print(torch_geometric.__version__)

import matplotlib.pyplot as plt
from torch_geometric.datasets import TUDataset
# from GraphRicciCurvature.FormanRicci import FormanRicci

import networkx as nx
import numpy as np
from scipy.sparse.csgraph import laplacian
from scipy.linalg import pinv, eigvalsh

from utils.load_data import *
from evaluation.metrics import *
from evaluation.metrics_distance import *
from evaluation.curvature import *
from visualization.plots import *
from visualization.networkx_plot import *


/usr/local/bin/python3


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/typing.py:128: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_scatter/_version_cpu.so, 0x0006): Symbol not found: __ZN5torch3jit17parseSchemaOrNameERKNSt3__112basic_stringIcNS1_11char_traitsIcEENS1_9allocatorIcEEEEb
  Referenced from: <39DD586D-4BCB-3117-B50F-6A4C67CC40D3> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_scatter/_version_cpu.so
  Expected in:     <F8622D92-25A9-3A61-A089-C917FDA36C1B> /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch_geometric/typing.py:139: UserWarning: An issue occurr

2.2.2
2.7.0


# Rewiring our graphs

🚩 We need to respect the experimental details of hyperparameters describe on page 15

## Dynamic Graph Rewiring Method Code

In [ ]:
from ipywidgets import widgets, Tab, VBox, Output

dataset_names = ["MUTAG"]  # Default dataset selection
REWIRING_METHOD = "BORF"   # Default rewiring method
ENTIRE_GRAPH = False       # Default entire graph selection
metrics = "Normal"         

output = Output()  



data = widgets.Dropdown(
    options=["REDDIT-BINARY", "IMDB-BINARY", "MUTAG", "ENZYMES", "PROTEINS"],
    value="MUTAG",
    description="Dataset:"
)

rewired_method = widgets.Dropdown(
    options=["BORF", "SDRF", "FOSR", "DES", "PPR", "LASER", "UNREWIRED"],
    value="BORF",
    description="Rewiring Method:"
)

entire_graph = widgets.Checkbox(
    value=False,
    description="Entire Graph"
)


metric = widgets.Dropdown(
    options=["Distance", "Normal"],
    value="Normal",
    description="Metrics:"
)


def update_dataset(change):
    global dataset_names
    dataset_names = [change["new"]]
    with output:
        output.clear_output()
        print(f"Dataset Selected: {dataset_names}")

def update_rewiring_method(change):
    global REWIRING_METHOD
    REWIRING_METHOD = change["new"]
    with output:
        output.clear_output()
        print(f"Rewiring Method Selected: {REWIRING_METHOD}")

def update_entire_graph(change):
    global ENTIRE_GRAPH
    ENTIRE_GRAPH = change["new"]
    with output:
        output.clear_output()
        print(f"Entire Graph Selected: {ENTIRE_GRAPH}")

def update_metrics(change):
    global metrics
    metrics = change["new"]
    with output:
        output.clear_output()
        print(f"Metrics Selected: {metrics}")


data.observe(update_dataset, names="value")
rewired_method.observe(update_rewiring_method, names="value")
entire_graph.observe(update_entire_graph, names="value")
metric.observe(update_metrics, names="value")


tab_contents = [
    VBox([data]),
    VBox([rewired_method]),
    VBox([entire_graph]),
    VBox([metric])
]

tab = Tab(children=tab_contents)

# Set tab titles
for i, title in enumerate(["Dataset", "Rewiring Method", "Entire Graph", "Metrics"]):
    tab.set_title(i, title)

display(tab, output)


Output()

In [13]:

print(f"Selected dataset: {dataset_names}")
print(f"Selected rewiring method: {REWIRING_METHOD}")
print(f"Selected entire graph: {ENTIRE_GRAPH}")
print(f"Selected metrics: {metrics}")

Selected dataset: ['PROTEINS']
Selected rewiring method: BORF
Selected entire graph: True
Selected metrics: Distance


In [12]:
from rewiring.rewiring_call import *
from tqdm import tqdm
import tkinter as tk
from tkinter import ttk, messagebox

# dataset_names = ["REDDIT-BINARY", "IMDB-BINARY", "MUTAG", "ENZYMES", "PROTEINS"]
#                   0              1            2         3           4
# dataset_names = dataset_names[2]

# REWIRING_METHOD = ["BORF","SDRF", "FOSR", "DES","PPR", "LASER", "UNREWIRED"]
# #                   0       1       2       3      4      5         6
# REWIRING_METHOD = REWIRING_METHOD[0]

# # flag all graph rewiring or not
# ENTIRE_GRAPH = True

dataset_loader = GraphDatasetLoader(dataset_names)
loaded_datasets = dataset_loader.get_loaded_dataset_names()

all_metrics_df = []

for dataset_name in loaded_datasets:
    for rewiring_name in [REWIRING_METHOD]:
        print("Rewiring methods being used:", rewiring_name)
        print(f"\n🚀 Processing dataset: {dataset_name}")
        
        # for testing purposes we can test on one graph
        if not ENTIRE_GRAPH:  
            # Get first graph
            graphs = dataset_loader.first_graphs[dataset_name]
            print("first graph", type(graphs))
        else:
            graphs = dataset_loader.datasets[dataset_name]  # Load full dataset

            
        for graph in tqdm(graphs):
            
            if isinstance(graphs, torch_geometric.data.data.Data):
                rewiring_method = rewiring_call(graphs, dataset_name)
            else:
                rewiring_method = rewiring_call(graph, dataset_name)
            
            if rewiring_name == "BORF":
                rewired_graph = rewiring_method.borf_rewiring()
            elif rewiring_name == "SDRF":
                rewired_graph = rewiring_method.sdrf_rewiring()
            elif rewiring_name == "FOSR":
                rewired_graph = rewiring_method.fosr_rewiring()
            elif rewiring_name == "LASER":
                rewired_graph = rewiring_method.laser_rewiring()
            elif rewiring_name == "DES":
                rewired_graph = rewiring_method.des_rewiring(dataset_loader)
            elif rewiring_name == "PPR":
                rewired_graph = rewiring_method.ppr_rewiring()
            elif rewiring_name == "UNREWIRED":
                
                G_nx = to_networkx(graph, to_undirected=True)  # Convert PyG graph to NetworkX
                print("G_nx", G_nx)
                rewired_graph = G_nx
           
            else:
                raise ValueError(f"Invalid rewiring method: {rewiring_name}")
            
            # Compute metrics for the rewired graph
            metrics_rewired = GraphMetrics(rewired_graph, dataset_name)
            df_metrics = metrics_rewired.get_all_metrics()
            df_metrics = pd.DataFrame([df_metrics]) 
            df_metrics["Rewiring Method"] = rewiring_name
            df_metrics["Dataset"] = dataset_name
            
            # Store and later save the metrics
            all_metrics_df.append(df_metrics)
    

# Convert results to DataFrame
final_df = pd.concat(all_metrics_df, ignore_index=True)

# Compute mean and standard deviation, excluding non-numeric columns
if ENTIRE_GRAPH:
    numeric_cols = final_df.select_dtypes(include=["number"])  
    avg_metrics = numeric_cols.mean().to_frame(name="Mean")  
    std_metrics = numeric_cols.std().to_frame(name="Std")  

    # Combine into a single DataFrame
    summary_df = pd.concat([avg_metrics, std_metrics], axis=1)

    # Format the output to display mean ± std in a single column
    summary_df["Formatted"] = summary_df.apply(lambda row: f"{row['Mean']:.6f} ± {row['Std']:.6f}", axis=1)

    # Save summary results correctly
    summary_output_csv = f"results/rewired_graph_avg_std_metrics_{dataset_name}_{REWIRING_METHOD}.csv"
    parent = Path(summary_output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    # If the path exists remove it
    if os.path.exists(summary_output_csv):
        os.remove(summary_output_csv)
    summary_df.to_csv(summary_output_csv, index=True)

    print(f"\n📂 Summary (Mean & Std) results saved to {summary_output_csv}.")
else:
    # Save individual rewiring results
    output_csv = f"results/rewired_graph_metrics_{dataset_name}.csv"
    # make sure the directory exists
    parent = Path(output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    final_df.to_csv(output_csv, index=False)
    print(f"\n📂 All rewiring results saved to {output_csv}.")

⚠️ Warning: 'ENZYMES' not found. Did you mean 'ENZYMES'?
✅ Dataset ENZYMES already exists. Loading from disk...
✅ Converted 600 graphs from ENZYMES into NetworkX format.
Rewiring methods being used: UNREWIRED

🚀 Processing dataset: ENZYMES
first graph <class 'torch_geometric.data.data.Data'>


  0%|          | 0/3 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'node_offsets'

In [14]:
from rewiring.rewiring_call import *
from tqdm import tqdm
import tkinter as tk
from tkinter import ttk, messagebox

# dataset_names = ["REDDIT-BINARY", "IMDB-BINARY", "MUTAG", "ENZYMES", "PROTEINS"]
#                   0              1            2         3           4
# dataset_names = dataset_names[2]

# REWIRING_METHOD = ["BORF","SDRF", "FOSR", "DES","PPR", "LASER", "UNREWIRED"]
# #                   0       1       2       3      4      5         6
# REWIRING_METHOD = REWIRING_METHOD[0]

# # flag all graph rewiring or not
# ENTIRE_GRAPH = True

dataset_loader = GraphDatasetLoader(dataset_names)
loaded_datasets = dataset_loader.get_loaded_dataset_names()

all_metrics_df = []
distance_metrics_df = []

for dataset_name in loaded_datasets:
    for rewiring_name in [REWIRING_METHOD]:
        print("Rewiring methods being used:", rewiring_name)
        print(f"\n🚀 Processing dataset: {dataset_name}")
        
        # for testing purposes we can test on one graph
        if not ENTIRE_GRAPH:  
            # Get first graph
            graphs = dataset_loader.first_graphs[dataset_name]
            # print("first graph", type(graphs))
        else:
            graphs = dataset_loader.datasets[dataset_name]  # Load full dataset

            
        for graph in tqdm(graphs):
            
            if isinstance(graphs, torch_geometric.data.data.Data):
                rewiring_method = rewiring_call(graphs, dataset_name)
            else:
                rewiring_method = rewiring_call(graph, dataset_name)
            
            if rewiring_name == "BORF":
                rewired_graph = rewiring_method.borf_rewiring()
            elif rewiring_name == "SDRF":
                rewired_graph = rewiring_method.sdrf_rewiring()
            elif rewiring_name == "FOSR":
                rewired_graph = rewiring_method.fosr_rewiring()
            elif rewiring_name == "LASER":
                rewired_graph = rewiring_method.laser_rewiring()
            elif rewiring_name == "DES":
                rewired_graph = rewiring_method.des_rewiring(dataset_loader)
            elif rewiring_name == "PPR":
                rewired_graph = rewiring_method.ppr_rewiring()
            elif rewiring_name == "UNREWIRED":
                
                #To account for if only using a single graph
                if isinstance(graphs, torch_geometric.data.data.Data):
                    G_nx = to_networkx(graphs, to_undirected=True)  # Convert PyG graph to NetworkX
                else:
                    G_nx = to_networkx(graph, to_undirected=True)
                print("G_nx", G_nx)
                rewired_graph = G_nx
           
            else:
                raise ValueError(f"Invalid rewiring method: {rewiring_name}")
            
            # Compute metrics for the rewired graph
            metrics_rewired = GraphMetrics(rewired_graph, dataset_name)
            df_metrics = metrics_rewired.get_all_metrics()
            df_metrics = pd.DataFrame([df_metrics]) 
            df_metrics["Rewiring Method"] = rewiring_name
            df_metrics["Dataset"] = dataset_name

            if metrics=="Distance":
                #Extra portion for distance metrics
                if isinstance(graphs, torch_geometric.data.data.Data):
                    G_orig = to_networkx(graphs, to_undirected=True)
                else:
                    G_orig = to_networkx(graph, to_undirected=True)
                distance_metrics_class = GraphDistanceMetrics(G_orig, rewired_graph, dataset_name)
                dist_metrics = distance_metrics_class.comparison_metrics()
                dist_metrics = pd.DataFrame([dist_metrics])
                dist_metrics["Rewiring Method"] = rewiring_name
                dist_metrics["Dataset"] = dataset_name

                distance_metrics_df.append(dist_metrics)

            
            # Store and later save the metrics
            all_metrics_df.append(df_metrics)
    

# Convert results to DataFrame
final_df = pd.concat(all_metrics_df, ignore_index=True)
#FOR DISTANCE METRICS
if metrics=="Distance":
    final_dist_df = pd.concat(distance_metrics_df, ignore_index=True)

# Compute mean and standard deviation, excluding non-numeric columns
if ENTIRE_GRAPH:
    numeric_cols = final_df.select_dtypes(include=["number"])  
    avg_metrics = numeric_cols.mean().to_frame(name="Mean")  
    std_metrics = numeric_cols.std().to_frame(name="Std")  

    # Combine into a single DataFrame
    summary_df = pd.concat([avg_metrics, std_metrics], axis=1)

    # Format the output to display mean ± std in a single column
    summary_df["Formatted"] = summary_df.apply(lambda row: f"{row['Mean']:.6f} ± {row['Std']:.6f}", axis=1)

    # Save summary results correctly
    summary_output_csv = f"results/rewired_graph_avg_std_metrics_{dataset_name}_{REWIRING_METHOD}.csv"
    parent = Path(summary_output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    # If the path exists remove it
    if os.path.exists(summary_output_csv):
        os.remove(summary_output_csv)
    summary_df.to_csv(summary_output_csv, index=True)

    print(f"\n📂 Summary (Mean & Std) results saved to {summary_output_csv}.")

    if metrics=="Distance":
        dist_num_cols = final_dist_df.select_dtypes(include=["number"])
        avg_dists = dist_num_cols.mean().to_frame(name="Mean")
        std_dists = dist_num_cols.std().to_frame(name="Std")

        summary_dists = pd.concat([avg_dists, std_dists], axis=1)

        summary_dists["Formatted"] = summary_dists.apply(lambda row: f"{row['Mean']:.6f} ± {row['Std']:.6f}", axis=1)

        summary_output_csv_dist = f"results/rewired_graph_DISTANCE_metrics_{dataset_name}_{REWIRING_METHOD}.csv"
        parent = Path(summary_output_csv_dist).parent
        os.makedirs(parent, exist_ok=True)

        if os.path.exists(summary_output_csv_dist):
            os.remove(summary_output_csv_dist)
        summary_dists.to_csv(summary_output_csv_dist, index=True)

        print(f"\n Distance Metrics results saved to {summary_output_csv_dist}.")


else:
    # Save individual rewiring results
    if metrics=="Normal":
        output_csv = f"results/rewired_graph_metrics_{dataset_name}.csv"
    else:
        output_csv = f"results/distance_metrics_{dataset_name}.csv"
    # make sure the directory exists
    parent = Path(output_csv).parent
    os.makedirs(parent, exist_ok=True)
    
    if metrics=="Normal":
        final_df.to_csv(output_csv, index=False)
    else:
        final_dist_df.to_csv(output_csv, index=False)
    print(f"\n📂 All rewiring results saved to {output_csv}.")

⚠️ Warning: 'PROTEINS' not found. Did you mean 'PROTEINS'?
✅ Dataset PROTEINS already exists. Loading from disk...
✅ Converted 1113 graphs from PROTEINS into NetworkX format.
Rewiring methods being used: BORF

🚀 Processing dataset: PROTEINS


  0%|          | 0/1113 [00:00<?, ?it/s]

[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...


  1%|          | 6/1113 [00:00<00:18, 59.35it/s]

[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...


...

[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additio

100%|█████████▉| 1110/1113 [00:18<00:00, 84.43it/s]

[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...
[INFO] Rewired graph for 50 iterations, 4 edge additio

100%|██████████| 1113/1113 [00:18<00:00, 61.45it/s]

[INFO] Rewired graph for 50 iterations, 4 edge additions and 2 edge removal exists...

📂 Summary (Mean & Std) results saved to results/rewired_graph_avg_std_metrics_PROTEINS_BORF.csv.

 Distance Metrics results saved to results/rewired_graph_DISTANCE_metrics_PROTEINS_BORF.csv.
